# Data Analytics in Healthcare

## Week 4 - Introduction to Spark

Objective: To learn about the Apache Spark platform and common abstractions and work with a simple example.

```
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.0
      /_/

Using Python version 2.7.6 (v2.7.6:3a1db0d2747e, Nov 10 2013 00:42:54)

```


In [1]:
# print Spark version
print("pyspark version: " + str(sc.version))

pyspark version: 2.1.0


#### Once you run the PySpark kernel, it defines a Spark context object (sc) and a Spark SQL Session class (SparkSession).

In [2]:
sc

In [3]:
ss = SparkSession(sc)
ss

## Part 1. Spark - RDD operations

The main abstraction Spark provides is a resilient distributed dataset (RDD), which is a collection of elements partitioned across the nodes of the cluster that can be operated on in parallel.

There are many ways to initialise an RDD.

In [4]:
rdd=sc.parallelize(range(1,1000))
rdd

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:475

#### Using ``` take ``` one can select the first few elements of an RDD

In [5]:
# take

x = sc.parallelize([1,2,3,4,5])
y = x.take(num = 3)
print(y)

# first
y = x.first()
print(y)

[1, 2, 3]


#### ```rdd.collect()``` converts a RDD object into a Python list on the host machine. i.e get all the values in an RDD. If the size of the values of the RDD is greater that the capacity of the host machine, this could result in failure.


In [6]:
# collect

x = sc.parallelize([1,2,3,4,5])
y = x.collect()
print(type(y))
print(y)  # not distributed

<type 'list'>
[1, 2, 3, 4, 5]


In [ ]:
# filter
y = x.filter(lambda x: x%2 == 1)  # filters out even elements
print(y.collect())

## Part 2: MapReduce

There are 2 basic functions in the MapReduce framework - **Map** and **Reduce**. The operations of Map & Reduce deal with two types: the type A of input data being mapped, and the type B of output data being reduced.

The **Map** function takes a series of key/value pairs, processes each, and generates zero or more output key/value pairs. The input and output types of the map can be (and often are) different from each other. Map operation takes individual values of type A and produces, for each a:A a value b:B; 

**Reduce** operation requires a binary operation • defined on values of type B; it consists of folding all available b:B to a single value.

In [14]:
# map

y = x.map(lambda x: (x,x**2))
print(x.collect())
print(y.collect())


# Flatmap - a function similar to map, introduced in Spark to allow a single list intead of list of tuples

# flatMap
y2 = x.flatMap(lambda x: (x, x**2))
print(y2.collect())

[1, 2, 3, 4, 5]
[(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)]
[1, 1, 2, 4, 3, 9, 4, 16, 5, 25]


In [23]:
# reduce

y = x.reduce(lambda obj, accumulated: obj + accumulated)  # computes a cumulative sum
print(y)

15


In [27]:
# reduceByKey
x = sc.parallelize([('B',1),('B',2),('A',3),('A',4),('A',5)])
y = x.reduceByKey(lambda v1, v2: v1 + v2)
print(y.collect())

[('A', 12), ('B', 3)]


## Part 3. Example - Word Count

Word Count is the "Hello, World!" of distributed data processing frameworks. We can readReading from Files

In [15]:
textRDD = sc.textFile("austen-sense.txt")


Each line is a separate element in the RDD

In [16]:
for line_no, element in enumerate(textRDD.take(10)):
    print line_no, element

0 [Sense and Sensibility by Jane Austen 1811]
1 
2 CHAPTER 1
3 
4 
5 The family of Dashwood had long been settled in Sussex.
6 Their estate was large, and their residence was at Norland Park,
7 in the centre of their property, where, for many generations,
8 they had lived in so respectable a manner as to engage
9 the general good opinion of their surrounding acquaintance.


In [17]:
# You can check the number of lines the text file has
textRDD.count()

14796

In [28]:
# Split each line into words
wordRDD = textRDD.flatMap(lambda line: line.split(" "))
print(wordRDD.take(5))

# Map each word to a single number
mappedwordsRDD = wordRDD.map(lambda word: (word, 1))

# Reduce by each word to find the word count of each number
wordCounts = mappedwordsRDD.reduceByKey(lambda word, count: word + count)

##Saving to Files


`.saveAsTextFile()` saves an RDD as a string. there is also `.saveAsPickleFile()`.

`.rsaveAsNewAPIHadoopDataset()` saves an RDD object to HDFS.

## Part 4 - Create a function

Create a function to accept a text file, and save a text file with the name "source_file_word_count.txt" with the word counts. Assume that the text file is too large to fit into memory, or the Hard Disk of your computer.


In [ ]:
def word_count(...):